In [ ]:
#importing the necessary libraries
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import Add, ReLU, Dense
from tensorflow.keras import Model

##### This is the basic building block. It contains a convolutional layer followed by a BatchNormalization which is followed by the ReLU activation function.

In [2]:
# Conv-BatchNorm-ReLU block (Convolutional layer followed by BatchNormalization followed by Relu activation function)
# x is the input and other factors are the other hyperparameters
def conv_batchnorm_relu(x, filters, kernel_size, strides = 1):
    x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

##### An identity block contains 3 convolutional layers, and each layer is followed by Batch Normalization and ReLU activation, except the last layer which first adds to the skip connection and only then applied the ReLU activation.Also, the first two convolutional layers have the same number of filters, but the last convolutional layer has 4 times the number of filters.

In [23]:
# Identity Block
def identity_block(tensor, filters):
    
    x = conv_batchnorm_relu(tensor, filters = filters,kernel_size = 1,strides=1)
    x = conv_batchnorm_relu(x,filters = filters,kernel_size = 3,strides=1)
    x = Conv2D(filters = 4*filters,kernel_size = 1, strides = 1)(x)# The last layer before adding skip connection
    x = BatchNormalization()(x)
    
    x = Add()([tensor,x])
    x = ReLU()(x)
    
    return x

In [24]:
# Projection Block
def projection_block(tensor, filters, strides):    
    #left stream
    x = conv_batchnorm_relu(tensor, filters=filters, kernel_size=1, strides=strides)
    x = conv_batchnorm_relu(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    #right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [25]:
# ResNet Block
def resnet_block(x,filters,reps,strides):
    x = projection_block(x, filters,strides)
    for _ in range(reps-1):
        x = identity_block(x,filters)
        
    return x

##### Now that all the blocks are ready, the model can now be created

In [26]:
# Creating the Model
input = Input(shape=(224,224,3))

x = conv_batchnorm_relu(input, filters=64, kernel_size=7, strides=2)
x = MaxPool2D(pool_size = 3, strides =2)(x)
x = resnet_block(x, filters=64, reps =3, strides=1)
x = resnet_block(x, filters=128, reps =4, strides=2)
x = resnet_block(x, filters=256, reps =6, strides=2)
x = resnet_block(x, filters=512, reps =3, strides=2)
x = GlobalAvgPool2D()(x)

output = Dense(1000, activation ='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 112, 112, 64) 9472        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_130 (BatchN (None, 112, 112, 64) 256         conv2d_130[0][0]                 
__________________________________________________________________________________________________
re_lu_119 (ReLU)                (None, 112, 112, 64) 0           batch_normalization_130[0][0]    
____________________________________________________________________________________________